In [164]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors


In [165]:
df_book=pd.read_csv("dataset/Book reviews/Book reviews/BX_Books.csv",sep=';',encoding = 'latin-1')
df_rating=pd.read_csv("dataset/Book reviews/Book reviews/BX-Book-Ratings.csv",sep=';',encoding = 'latin-1')
# df_processed=pd.read_csv("dataset/Books Data with Category Language and Summary/Preprocessed_data.csv",sep=';',encoding = 'latin-1')


In [166]:
df_rating['Book-Rating'].value_counts()

0     716109
8     103736
10     78610
7      76457
9      67541
5      50974
6      36924
4       8904
3       5996
2       2759
1       1770
Name: Book-Rating, dtype: int64

In [167]:
# Replace rating 0 into NaN
df_rating.replace(0,np.nan,inplace=True)

In [168]:
df_rating['Book-Rating'].value_counts()


8.000     103736
10.000     78610
7.000      76457
9.000      67541
5.000      50974
6.000      36924
4.000       8904
3.000       5996
2.000       2759
1.000       1770
Name: Book-Rating, dtype: int64

In [170]:
user_book_rating=pd.merge(df_book,df_rating,on="ISBN")
print(user_book_rating.isnull().sum())
user_book_rating['User-ID'].value_counts()

ISBN                        0
Book-Title                  0
Book-Author                 1
Year-Of-Publication         0
Publisher                   2
Image-URL-S                 0
Image-URL-M                 0
Image-URL-L                 0
User-ID                     0
Book-Rating            647323
dtype: int64


11676     11144
198711     6456
153662     5814
98391      5779
35859      5646
          ...  
72897         1
51911         1
151772        1
158802        1
259920        1
Name: User-ID, Length: 92107, dtype: int64

In [171]:
len(user_book_rating['Book-Title'].unique())

241090

In [172]:
# total rating count
book_ratingCount = (user_book_rating.
     groupby(by = ['Book-Title'])['Book-Rating'].
     count().
     reset_index().
     rename(columns = {'Book-Rating': 'totalRatingCount'})
     [['Book-Title', 'totalRatingCount']]
    )
book_ratingCount.head()

,Book-Title,totalRatingCount
0,A Light in the Storm: The Civil War Diary of ...,1
1,Always Have Popsicles,0
2,Apple Magic (The Collector's series),0
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,0


In [173]:
rating_with_totalRatingCount = user_book_rating.merge(book_ratingCount, left_on = 'Book-Title', right_on = 'Book-Title', how = 'left')
rating_with_totalRatingCount.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,Book-Rating,totalRatingCount
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,2,NaN,1
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,8,5.000,9
2,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11400,NaN,9
3,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11676,8.000,9
4,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,41385,NaN,9


In [174]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
print(book_ratingCount['totalRatingCount'].describe())

count   241090.000
mean         1.592
std          6.993
min          0.000
25%          0.000
50%          1.000
75%          1.000
max        707.000
Name: totalRatingCount, dtype: float64


In [175]:
# Extract books with totalRatingCount>=50
popularity_threshold = 50
rating_popular_book = rating_with_totalRatingCount.query('totalRatingCount >= @popularity_threshold')
rating_popular_book.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,Book-Rating,totalRatingCount
30,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,8,NaN,116
31,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,11676,9.000,116
32,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,29526,9.000,116
33,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,36836,NaN,116
34,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,http://images.amazon.com/images/P/0399135782.0...,46398,9.000,116


In [176]:
# Show value count on features
print(rating_popular_book['User-ID'].value_counts())
print(rating_popular_book['Book-Title'].value_counts())

11676     886
35859     402
16795     315
238120    304
153662    286
         ... 
32942       1
1108        1
271471      1
271087      1
228196      1
Name: User-ID, Length: 35223, dtype: int64
Wild Animus                                                     2502
The Lovely Bones: A Novel                                       1295
The Da Vinci Code                                                898
A Painted House                                                  838
The Nanny Diaries: A Novel                                       828
                                                                ... 
Jitterbug Perfume                                                 88
Hearts In Atlantis : New Fiction                                  87
The Laughing Corpse (Anita Blake Vampire Hunter (Paperback))      87
The Hobbit                                                        80
Free                                                              56
Name: Book-Title, Length: 651, dtype: int64


In [177]:
# Create pivot table (Book-Title/User-ID) filled with rating values

book_user_rating=rating_popular_book.pivot_table('Book-Rating',index='Book-Title',columns='User-ID')
book_user_rating.head()

User-ID,9,16,26,32,42,51,91,97,99,114,...,278800,278807,278824,278828,278832,278836,278843,278844,278846,278854
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1st to Die: A Novel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2nd Chance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4 Blondes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84 Charing Cross Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [178]:

print(len(book_user_rating))
print(len(book_user_rating.columns))

651
24863


In [180]:
# Fill NAN into 0.0
book_user_rating.fillna(0,inplace=True)
book_user_rating.head(3)

User-ID,9,16,26,32,42,51,91,97,99,114,...,278800,278807,278824,278828,278832,278836,278843,278844,278846,278854
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1st to Die: A Novel,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2nd Chance,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [181]:
similarity_matrix=cosine_similarity(book_user_rating)
similarity_matrix=pd.DataFrame(data=similarity_matrix,index=book_user_rating.index,columns=book_user_rating.index)
similarity_matrix.head()

Book-Title,1984,1st to Die: A Novel,2nd Chance,4 Blondes,84 Charing Cross Road,A Beautiful Mind: The Life of Mathematical Genius and Nobel Laureate John Nash,A Bend in the Road,A Case of Need,"A Child Called \It\"": One Child's Courage to Survive""",A Confederacy of Dunces (Evergreen Book),...,Wifey,Wild Animus,Winter Solstice,Wish You Well,Without Remorse,"Wizard and Glass (The Dark Tower, Book 4)",Wuthering Heights,Year of Wonders,Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,"\O\"" Is for Outlaw"""
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,1.000,0.023,0.007,0.000,0.033,0.014,0.013,0.041,0.019,0.022,...,0.026,0.018,0.014,0.020,0.011,0.049,0.028,0.023,0.039,0.016
1st to Die: A Novel,0.023,1.000,0.163,0.000,0.009,0.000,0.032,0.027,0.030,0.006,...,0.025,0.009,0.035,0.061,0.035,0.025,0.025,0.019,0.014,0.056
2nd Chance,0.007,0.163,1.000,0.011,0.009,0.000,0.042,0.047,0.000,0.018,...,0.051,0.025,0.033,0.049,0.009,0.049,0.011,0.017,0.009,0.033
4 Blondes,0.000,0.000,0.011,1.000,0.000,0.000,0.000,0.037,0.000,0.012,...,0.031,0.002,0.000,0.000,0.000,0.003,0.023,0.025,0.010,0.000
84 Charing Cross Road,0.033,0.009,0.009,0.000,1.000,0.021,0.008,0.029,0.010,0.039,...,0.023,0.025,0.021,0.022,0.014,0.025,0.017,0.000,0.011,0.035


In [182]:
similarity_matrix['1st to Die: A Novel'].sort_values(ascending=False)[:5]

Book-Title
1st to Die: A Novel                 1.000
2nd Chance                          0.163
Roses Are Red (Alex Cross Novels)   0.143
Violets Are Blue                    0.138
Kiss the Girls                      0.126
Name: 1st to Die: A Novel, dtype: float64

In [183]:
# Use KNN algorithm with cosine similarity metric
knn=NearestNeighbors(metric='cosine',algorithm='brute')
knn.fit(book_user_rating.values)
distances, indices=knn.kneighbors(book_user_rating.values,n_neighbors=3)
print(indices)
print(distances)

[[  0  51  85]
 [  1   2 371]
 [  2   1 620]
 ...
 [648 292 593]
 [649  57 185]
 [650 210  92]]
[[4.44089210e-16 8.44175106e-01 8.63022413e-01]
 [0.00000000e+00 8.36555091e-01 8.56674030e-01]
 [0.00000000e+00 8.36555091e-01 8.40768556e-01]
 ...
 [3.33066907e-16 9.19586846e-01 9.24079319e-01]
 [0.00000000e+00 9.07466512e-01 9.29253912e-01]
 [3.33066907e-16 8.04459732e-01 8.15560032e-01]]


In [184]:
# copy df
df = book_user_rating.copy()
predicted_df = book_user_rating.copy()
predicted_df.head()

User-ID,9,16,26,32,42,51,91,97,99,114,...,278800,278807,278824,278828,278832,278836,278843,278844,278846,278854
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1st to Die: A Novel,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2nd Chance,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
4 Blondes,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
84 Charing Cross Road,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [185]:

def itembased_predict_rating(user_id,number_neighbors):
  
  # find the nearest neighbors using KNN
  # compute similarity with cosine simularity
  knn = NearestNeighbors(metric='cosine', algorithm='brute')
  knn.fit(df.values)
  distances, indices = knn.kneighbors(df.values, n_neighbors=number_neighbors)

  # convert user_id to user_index
  user_index = df.columns.tolist().index(user_id)

  # t: book_title, m: the row number of t in df
  for m,t in list(enumerate(df.index)):
    
    # find books without ratings by target user
    if df.iloc[m, user_index] == 0:
      sim_books = indices[m].tolist()
      book_distances = distances[m].tolist()
      
      # Generally, this is the case: indices[3] = [3 6 7]. The book itself is in the first place.
      # In this case, we take off 3 from the list. Then, indices[3] == [6 7] to have the nearest NEIGHBORS in the list. 
      if m in sim_books:
        id_book = sim_books.index(m)
        sim_books.remove(m)
        book_distances.pop(id_book) 

      # However, if the percentage of ratings in the dataset is very low, there are too many 0s in the dataset. 
      # Some books have all 0 ratings and the books with all 0s are considered the same books by NearestNeighbors(). 
      # Then,even the book itself cannot be included in the indices. 
      # For example, indices[3] = [2 4 7] is possible if book_2, book_3, book_4, and book_7 have all 0s for their ratings.
      # In that case, we take off the farthest book in the list. Therefore, 7 is taken off from the list, then indices[3] == [2 4].
      else:
        sim_books = sim_books[:number_neighbors-1]
        book_distances = book_distances[:number_neighbors-1]
          
      # book_similarty = 1 - book_distance    
      book_similarity = [1-x for x in book_distances]
      book_similarity_copy = book_similarity.copy()
      nominator = 0

      # for each similar book
      for s in range(0, len(book_similarity)):
        
        # check if the rating of a similar book is zero
        if df.iloc[sim_books[s], user_index] == 0:

          # if the rating is zero, ignore the rating and the similarity in calculating the predicted rating
          if len(book_similarity_copy) == (number_neighbors - 1):
            book_similarity_copy.pop(s)
            
          else:
            book_similarity_copy.pop(s-(len(book_similarity)-len(book_similarity_copy)))

        # if the rating is not zero, use the rating and similarity in the calculation
        else:
          nominator = nominator + book_similarity[s]*df.iloc[sim_books[s],user_index]

      # check if the number of the ratings with non-zero is positive
      if len(book_similarity_copy) > 0:
        
        # check if the sum of the ratings of the similar books is positive.
        if sum(book_similarity_copy) > 0:
          predicted_r = nominator/sum(book_similarity_copy)

        # Even if there are some books for which the ratings are positive, some books have zero similarity even though they are selected as similar books.
        # in this case, the predicted rating becomes zero as well  
        else:
          predicted_r = 0

      # if all the ratings of the similar books are zero, then predicted rating should be zero
      else:
        predicted_r = 0

    # place the predicted rating into the copy of the original dataset
      predicted_df.iloc[m,user_index] = predicted_r

In [186]:
def recommend_books(user, num_recommended_books):

  itembased_predict_rating(user,num_recommended_books)
  
  print('The list of the Movies User{} Has Watched \n'.format(user))

  for m in df[df[user] > 0][user].index.tolist():
    print(m)
  
  print('\n')

  recommended_books = []

  for m in df[df[user] == 0].index.tolist():

    index_df = df.index.tolist().index(m)
    predicted_rating = predicted_df.iloc[index_df, predicted_df.columns.tolist().index(user)]
    recommended_books.append((m, predicted_rating))

  sorted_rm = sorted(recommended_books, key=lambda x:x[1], reverse=True)
  
  print('The list of the Recommended Books \n')
  rank = 1
  for recommended_book in sorted_rm[:num_recommended_books]:
    
    print('{}: {} - predicted rating:{}'.format(rank, recommended_book[0], recommended_book[1]))
    rank = rank + 1

In [199]:
recommend_books(153662, 5)

The list of the Movies User153662 Has Watched 

A Separate Peace
A Time to Kill
A Wrinkle in Time
Anne of Green Gables (Anne of Green Gables Novels (Paperback))
Beloved
Birthright
Breath, Eyes, Memory
Breathing Lessons
Bridget Jones's Diary
Carolina Moon
Chicken Soup for the Woman's Soul (Chicken Soup for the Soul Series (Paper))
Christine
Cujo
Daisy Fay and the Miracle Man
Dance upon the Air (Three Sisters Island Trilogy)
Ellen Foster
Face the Fire (Three Sisters Island Trilogy)
Fahrenheit 451
Girl in Hyacinth Blue
Harry Potter and the Chamber of Secrets (Book 2)
Harry Potter and the Goblet of Fire (Book 4)
Harry Potter and the Order of the Phoenix (Book 5)
Harry Potter and the Prisoner of Azkaban (Book 3)
Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))
Heart of the Sea (Irish Trilogy)
Heartbreaker
Heaven and Earth (Three Sisters Island Trilogy)
House of Sand and Fog
I Capture the Castle
Illusions: The Adventures of a Reluctant Messiah
Intensity
It
Jewels of the Sun (